### Notebook for Project2 for Programming for Data Analysis

## An analysis of paleo-present climate data

***

## Table of Contents

1. [Introduction](#Introduction)
2. 

### CO2 vs Temperatue Anomaly from 800kyrs - present



---

### Examine one other feature - CH4 for example



---

### Examine Irish Context



---

### Fuse and analysis of data from various data sources and Export to csv and json



---

### Analysis of all the above data - trends/relationships etc (temporal leads, lags, frequency analysis etc)



---

### Predict global temperature anomaly over next few decades (synthesise data) and compare to pushed climate modles - if atmospheric CO2 trends continue



---

### Comment on accellerated warming based on very latest features (e.g temperature/polar-ice-coverage)



---

### Appendix/References etc

***

#### End

Notes:

In [3]:
# json

# Reading from a json file
'''
import json

FILENAME = "json_text.txt"

with open(FILENAME, 'rt') as f:
    dict = json.load(f)

print (dict["name"], dict ["age"])

print (dict)
'''


# Writing to a json file
'''
import json

FILENAME = "json_text.txt"

dict = {}
dict["name"] = input("Enter a name: ")
dict["age"] = int(input("Enter an age: "))
print (dict)

#1 - assume the file exists
with open(FILENAME, 'wt') as f:
    json.dump(dict, f)
'''

'\nimport json\n\nFILENAME = "json_text.txt"\n\ndict = {}\ndict["name"] = input("Enter a name: ")\ndict["age"] = int(input("Enter an age: "))\nprint (dict)\n\n#1 - assume the file exists\nwith open(FILENAME, \'wt\') as f:\n    json.dump(dict, f)\n'